In [192]:
import hanabi_multiagent_framework as hmf
from hanabi_multiagent_framework.utils import make_hanabi_env_config
import numpy as np
from numpy import ndarray
from hanabi_agents.rule_based import ParallelRulebasedAgent
import hanabi_agents.rule_based.predefined_rules as rules
from hanabi_learning_environment import pyhanabi, rl_env
from hanabi_agents.rule_based.ruleset import Ruleset 
from evolve import Evolution
import timeit
import statistics
import dill as pickle
import math

## Final Entropy Code

In [189]:
def Intra_Agent_Entropy(my_rules):

    names_dict = {}

    for i in rule_names:
        names_dict[i] = 1e-6 

    agent_rules = []
    #For all agents, make a names dictionary
    for _ in range(my_rules.shape[0]):
        agent_rules.append(names_dict.copy())


    agent_rules = np.array(agent_rules) 

    for i in range(my_rules.shape[0]):
        rule_list = my_rules[i]
        agent = agent_rules[i]
        for rule in rule_list:
            if agent[rule.__name__] == 1e-6:
                agent[rule.__name__] = 1

            else:
                agent[rule.__name__] += 1

    # Update Probabilities
    agent_probs = []

    for agent in agent_rules:
        agent_probs.append(agent.copy())

    for agent in agent_probs:
        for rule in agent.keys():
            agent[rule] = agent[rule] / my_rules.shape[1]


    # Calculate Entropy
    Entropies = []
    for agent in agent_probs:
        E = 0
        for rules in agent.keys():
            E += - agent[rule] * math.log(agent[rule])

        Entropies.append(E)

    Entropies = np.array(Entropies)

    return Entropies

In [207]:
population_size = 5
n_rules = 20
my_rules = np.array([rules.random_rules(n_rules) for _ in range(population_size)])

# diversity = Intra_Agent_Entropy(my_rules)

In [226]:
rule_names = []
for i in range(my_rules.shape[0]):
    for j in range(my_rules.shape[1]):
        rule_names.append(my_rules[i,j].__name__)

rule_names = np.unique(rule_names)

(12,)

array(['discard_oldest_first', 'discard_randomly', 'hail_mary',
       'osawa_discard', 'play_probably_safe_treshold',
       'play_probably_useless_treshold', 'tell_anyone_useful_card',
       'tell_anyone_useless_card', 'tell_dispensable',
       'tell_most_information', 'tell_playable_card_outer',
       'tell_randomly'], dtype='<U30')

In [212]:
diversity

array([2, 3, 0, 1, 4])

In [215]:
rule_names

array(['discard_oldest_first', 'discard_randomly', 'hail_mary',
       'osawa_discard', 'play_probably_safe_treshold',
       'play_probably_useless_treshold', 'tell_anyone_useful_card',
       'tell_anyone_useless_card', 'tell_dispensable',
       'tell_most_information', 'tell_playable_card_outer',
       'tell_randomly'], dtype='<U30')